![example](images/director_shot.jpeg)

# Project Title

**Authors:** Student 1, Student 2, Student 3
***

## Overview

A one-paragraph overview of the project, including the business problem, data, methods, results and recommendations.

## Business Problem

Summary of the business problem you are trying to solve, and the data questions that you plan to answer to solve them.

***
Questions to consider:
* What are the business's pain points related to this project?
* How did you pick the data analysis question(s) that you did?
* Why are these questions important from a business perspective?
***

## Data Understanding

Describe the data being used for this project.
***
Questions to consider:
* Where did the data come from, and how do they relate to the data analysis questions?
* What do the data represent? Who is in the sample and what variables are included?
* What is the target variable?
* What are the properties of the variables you intend to use?
***

In [1]:
# Import standard packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [3]:
import glob, os

In [4]:
fpath='zippedData/'
os.listdir(fpath)

['imdb.title.crew.csv.gz',
 'tmdb.movies.csv.gz',
 'imdb.title.akas.csv.gz',
 'imdb.title.ratings.csv.gz',
 'imdb.name.basics.csv.gz',
 'rt.reviews.tsv.gz',
 'imdb.title.basics.csv.gz',
 'rt.movie_info.tsv.gz',
 'tn.movie_budgets.csv.gz',
 'bom.movie_gross.csv.gz',
 'imdb.title.principals.csv.gz']

In [6]:
query=fpath+'*.gz'
file_list = glob.glob(query)
file_list

['zippedData/imdb.title.crew.csv.gz',
 'zippedData/tmdb.movies.csv.gz',
 'zippedData/imdb.title.akas.csv.gz',
 'zippedData/imdb.title.ratings.csv.gz',
 'zippedData/imdb.name.basics.csv.gz',
 'zippedData/rt.reviews.tsv.gz',
 'zippedData/imdb.title.basics.csv.gz',
 'zippedData/rt.movie_info.tsv.gz',
 'zippedData/tn.movie_budgets.csv.gz',
 'zippedData/bom.movie_gross.csv.gz',
 'zippedData/imdb.title.principals.csv.gz']

In [20]:
tables={}

In [21]:
for file in file_list:
    print('---'*20)
    file_name=file.replace('zippedData/','').replace('.','_')
    print(file_name)
    if '.tsv.gz' in file:
        temp_df = pd.read_csv(file, sep='\t', encoding='latin-1')
    else:
        temp_df=pd.read_csv(file)
    display(temp_df.head(),temp_df.tail())
    tables[file_name]=temp_df
    

------------------------------------------------------------
imdb_title_crew_csv_gz


,tconst,directors,writers
0,tt0285252,nm0899854,nm0899854
1,tt0438973,NaN,"nm0175726,nm1802864"
2,tt0462036,nm1940585,nm1940585
3,tt0835418,nm0151540,"nm0310087,nm0841532"
4,tt0878654,"nm0089502,nm2291498,nm2292011",nm0284943


,tconst,directors,writers
146139,tt8999974,nm10122357,nm10122357
146140,tt9001390,nm6711477,nm6711477
146141,tt9001494,"nm10123242,nm10123248",NaN
146142,tt9004986,nm4993825,nm4993825
146143,tt9010172,NaN,nm8352242


------------------------------------------------------------
tmdb_movies_csv_gz


,Unnamed: 0,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count
0,0,"[12, 14, 10751]",12444,en,Harry Potter and the Deathly Hallows: Part 1,33.533,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788
1,1,"[14, 12, 16, 10751]",10191,en,How to Train Your Dragon,28.734,2010-03-26,How to Train Your Dragon,7.7,7610
2,2,"[12, 28, 878]",10138,en,Iron Man 2,28.515,2010-05-07,Iron Man 2,6.8,12368
3,3,"[16, 35, 10751]",862,en,Toy Story,28.005,1995-11-22,Toy Story,7.9,10174
4,4,"[28, 878, 12]",27205,en,Inception,27.920,2010-07-16,Inception,8.3,22186


...

,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000,2010
3,Inception,WB,292600000.0,535700000,2010
4,Shrek Forever After,P/DW,238700000.0,513900000,2010


,title,studio,domestic_gross,foreign_gross,year
3382,The Quake,Magn.,6200.0,NaN,2018
3383,Edward II (2018 re-release),FM,4800.0,NaN,2018
3384,El Pacto,Sony,2500.0,NaN,2018
3385,The Swan,Synergetic,2400.0,NaN,2018
3386,An Actor Prepares,Grav.,1700.0,NaN,2018


------------------------------------------------------------
imdb_title_principals_csv_gz


,tconst,ordering,nconst,category,job,characters
0,tt0111414,1,nm0246005,actor,NaN,"[""The Man""]"
1,tt0111414,2,nm0398271,director,NaN,NaN
2,tt0111414,3,nm3739909,producer,producer,NaN
3,tt0323808,10,nm0059247,editor,NaN,NaN
4,tt0323808,1,nm3579312,actress,NaN,"[""Beth Boothby""]"


,tconst,ordering,nconst,category,job,characters
1028181,tt9692684,1,nm0186469,actor,NaN,"[""Ebenezer Scrooge""]"
1028182,tt9692684,2,nm4929530,self,NaN,"[""Herself"",""Regan""]"
1028183,tt9692684,3,nm10441594,director,NaN,NaN
1028184,tt9692684,4,nm6009913,writer,writer,NaN
1028185,tt9692684,5,nm10441595,producer,producer,NaN


In [22]:
tables.keys()

dict_keys(['imdb_title_crew_csv_gz', 'tmdb_movies_csv_gz', 'imdb_title_akas_csv_gz', 'imdb_title_ratings_csv_gz', 'imdb_name_basics_csv_gz', 'rt_reviews_tsv_gz', 'imdb_title_basics_csv_gz', 'rt_movie_info_tsv_gz', 'tn_movie_budgets_csv_gz', 'bom_movie_gross_csv_gz', 'imdb_title_principals_csv_gz'])

# Approach #

The main variables I will need are title, genre, language, production company, budget, box office gross, and release date. All of these are available on IMDB and using the IMDB id will allow for more accurate joins.  I will still explore whether there is better/additional information for some of the variables available from other sources, especially for variables like budget which are harder to find.

I am going to start with the imdb_title_basics_csv_gz data set, since it has the most movies listed with id, title, and genre information. It also has the year of earliest release.  This will allow me to filter movies by year.  

For this analysis, I am only interested in movies released in the past 10 years, since this information will be most relevant to any recommendations made to Microsoft for future projects. I am going back 10 years since this is an analysis mainly using box office gross and the past two years will be unreliable due to a shift to streaming services caused by the COVID-19 pandemic. While it is unclear if this will be a permanent shift in the film industry, we will be specifically making recommendations for movies to be released in theaters and therefore want to include as much relevant data as possible.

In [32]:
#Filter out all films in imdb_titles_basic that were made released prior to 2011.
#start_year is int

imdb_titles=tables['imdb_title_basics_csv_gz']
print('Rows with all movies: ' + str(imdb_titles.shape[0]))

imdb_10yr=imdb_titles[imdb_titles['start_year']<2011]
print('Rows with movies in the past 10 years: ' + str(imdb_10yr.shape[0]))

Rows with all movies: 146144
Rows with movies in the past 10 years: 11849


The next variable I will try to join is language, since I will be filtering for all movies made in English. While Microsoft could produce foreign language movies, we are limited this analysis to just those produced in English.

I will use the IMDB get_versions endpoint to find the primary spoken language for each film.

In [33]:
#test endpoint use with first five movies
first_five=imdb_10yr[:5]
first_five

,tconst,primary_title,original_title,start_year,runtime_minutes,genres
10,tt0146592,Pál Adrienn,Pál Adrienn,2010,136.0,Drama
11,tt0154039,So Much for Justice!,Oda az igazság,2010,100.0,History
13,tt0162942,Children of the Green Dragon,A zöld sárkány gyermekei,2010,89.0,Drama
18,tt0230212,The Final Journey,The Final Journey,2010,120.0,Drama
37,tt0312305,Quantum Quest: A Cassini Space Odyssey,Quantum Quest: A Cassini Space Odyssey,2010,45.0,"Adventure,Animation,Sci-Fi"


In [54]:
for i in first_five['tconst'].iteritems():
    print(i[1])

tt0146592
tt0154039
tt0162942
tt0230212
tt0312305


In [36]:
import requests
import json

In [38]:
#load key for IMDB API
def get_keys(path):
    with open(path) as f:
        return json.load(f)
    
keys = get_keys("/Users/morgandj/.secret/imdb_creds.json")
host=keys['host']
key=keys['key']

In [69]:
get_versions_url='https://imdb8.p.rapidapi.com/title/get-versions'
headers={'x-rapidapi-host': host, 
         'x-rapidapi-key': key}
responses=[]

for i in first_five['tconst'].iteritems():
    imdb_id=i[1]
    params={'tconst': imdb_id}
    response=requests.get(get_versions_url,headers=headers, params=params)
    responses.append(response.json())

In [76]:
responses[0]

{'@type': 'imdb.api.title.versions',
 'alternateTitles': [{'region': 'PL',
   'title': 'Adrienn Pal',
   'types': ['imdbDisplay']},
  {'language': 'bg',
   'region': 'BG',
   'title': 'Адриен Пал',
   'types': ['imdbDisplay']},
  {'region': 'HR', 'title': 'Pal Adrienn', 'types': ['imdbDisplay']},
  {'region': 'HU', 'title': 'Pál Adrienn', 'types': ['imdbDisplay']},
  {'region': 'SE', 'title': 'Adrienn Pál', 'types': ['imdbDisplay']},
  {'title': 'Pál Adrienn', 'types': ['original']}],
 'colorations': ['color'],
 'defaultTitle': 'Pál Adrienn',
 'silent': False,
 'spokenLanguages': ['hu'],
 'originalTitle': 'Pál Adrienn',
 'origins': ['HU', 'FR', 'AT', 'NL'],
 'runningTimes': [{'timeMinutes': 136}],
 'id': '/title/tt0146592/',
 'title': 'Pál Adrienn',
 'titleType': 'movie',
 'year': 2010}

The 'origins' key will be used to find films originating in the US, since Microsoft's studio will be in the US. 

In [72]:
countries=[]
for r in responses:
    country=r['origins'][0]
    countries.append(country)
    
countries


['HU', 'HU', 'HU', 'US', 'US']

In [75]:
first_five.loc[:,'country']=countries
first_five

/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/pandas/core/indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


,tconst,primary_title,original_title,start_year,runtime_minutes,genres,country
10,tt0146592,Pál Adrienn,Pál Adrienn,2010,136.0,Drama,HU
11,tt0154039,So Much for Justice!,Oda az igazság,2010,100.0,History,HU
13,tt0162942,Children of the Green Dragon,A zöld sárkány gyermekei,2010,89.0,Drama,HU
18,tt0230212,The Final Journey,The Final Journey,2010,120.0,Drama,US
37,tt0312305,Quantum Quest: A Cassini Space Odyssey,Quantum Quest: A Cassini Space Odyssey,2010,45.0,"Adventure,Animation,Sci-Fi",US


In [77]:
imdb_10=imdb_10yr.copy()

In [81]:
version_responses=[]
for i in imdb_10['tconst'].iteritems():
    imdb_id=i[1]
    #print(imdb_id)
    params={'tconst': imdb_id}
    response=requests.get(get_versions_url,headers=headers, params=params)
    version_responses.append(response)
print(len(version_responses))
version_responses[:2]

KeyboardInterrupt: 

In [84]:
#function that calls data on a list of movie ids from a specific endpoint in imdb
#returns a list of responses
def imdb_requests_byid(list_ids,url):
    responses=[]
    for i in list_ids:
        imdb_id=i
        #print(imdb_id)
        params={'tconst': imdb_id}
        response=requests.get(url,headers=headers, params=params)
        responses.append(response)
    return responses


In [87]:
#make list of all ids for movies from last 10 years
imdb_10_ids=imdb_10['tconst']
id_list=imdb_10_ids.tolist()
len(id_list)

11849

In [97]:
#break the longer list into 500 id chunks to help me see where the errors are
#wasn't able to run query on everything at once
list_of_lists=[id_list[(i-500):i] for i in list(range(500,12500,500))]
len(list_of_lists[-1])


349

In [108]:
import time

In [110]:
#Loop through all of the lists and make requests 500 at a time
version_response_lists=[]
for l in list_of_lists:
    print(l[0])
    version_responses=imdb_requests_byid(l,get_versions_url)
    version_response_lists.append(version_responses)
len(version_response_lists)

tt0146592
tt1196141
tt1332570
tt1442223
tt1513829
tt1554086
tt1587811
tt1611220
tt1633334
tt1655562
tt1675846
tt1700275
tt1723051
tt1742074
tt1764388
tt1785498
tt1810858
tt1846675
tt1931482
tt2085919
tt2391829
tt3378222
tt4742256
tt7015734


24

In [112]:
versions_json_list=[]
error_list=[]
for v in version_response_lists:
    json_list=[]
    errors=[]
    for r in v:
        if r.status_code == 200:
            json_list.append(r.json())
        else: 
            errors.append(r.status_code)
    versions_json_list.extend(json_list)
    error_list.extend(errors)
    
print(len(versions_json_list))
error_list
    

11830


[503,
 503,
 503,
 503,
 503,
 503,
 503,
 503,
 503,
 503,
 503,
 503,
 503,
 503,
 503,
 503,
 503,
 503,
 503]

Error 503 means the server is currently unable to handle the http request and to try again later.  As I go through the requests, any movies I don't get because of this error can be tried again later.  

In [119]:
#create a list of dictionaries with keys tconst and country
#convert into df

country_dictionaries=[]

for v in versions_json_list:
    if (v != None) and ('origins' in v.keys()) and ('id' in v.keys()):
        country_dict={}
        tconst=v['id']
        if tconst != None:
            country_dict['tconst']=tconst[7:16]
        else:
            country_dict['tconst']=None
        country=v['origins']
        if country != None:
            country_dict['country']=country[0]
        else:
            country_dict['country']=None
        country_dictionaries.append(country_dict)
    
country_df=pd.DataFrame(country_dictionaries)
country_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11687 entries, 0 to 11686
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   tconst   11687 non-null  object
 1   country  11687 non-null  object
dtypes: object(2)
memory usage: 182.7+ KB


We tried to make 11849 requests and have 11687 movies that we were able to get both id and country for. This means about 1% of the original list will be dropped when we join countries if we do an inner join. This will be acceptable- I would rather ensure I only include movies made in the US.

In [120]:
#see how many movies were made in the US
country_df['country'].value_counts()

US    4184
IN     676
GB     570
FR     519
CA     518
      ... 
SZ       1
KP       1
AL       1
SA       1
JM       1
Name: country, Length: 148, dtype: int64

4184 movies were made in the US. We will drop the movies that did not originate from the US and then join this dataframe with the dataframe of movies made in the last 10 years.

In [121]:
#filter for the movies originating in the US
us_df=country_df[country_df['country']=='US']
us_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4184 entries, 3 to 11676
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   tconst   4184 non-null   object
 1   country  4184 non-null   object
dtypes: object(2)
memory usage: 98.1+ KB


In [122]:
#merge the dataframes, inner join to only include US movies
imdb_10_US=imdb_10.merge(us_df,how='inner',on='tconst')
imdb_10_US.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4166 entries, 0 to 4165
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   tconst           4166 non-null   object 
 1   primary_title    4166 non-null   object 
 2   original_title   4166 non-null   object 
 3   start_year       4166 non-null   int64  
 4   runtime_minutes  3684 non-null   float64
 5   genres           4096 non-null   object 
 6   country          4166 non-null   object 
dtypes: float64(1), int64(1), object(5)
memory usage: 260.4+ KB


Now that we have a list of movies made in the US in the past 10 years, I'll query the IMDB API end point get-business for budget and box office data.  We can also see that some of the genre data is missing.  For now I'll still get the info for those movies, since we may do analysis that doesn't include genre. For this project, I am not intending on using runtime, so I am not concerned that some of that data is missing.

In [128]:
ids_10_US=imdb_10_US['tconst'].tolist()

In [129]:
imdb_business_url='https://imdb8.p.rapidapi.com/title/get-business'
bus_responses=imdb_requests_byid(ids_10_US,imdb_business_url)
bus_responses

[<Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [

In [134]:
bus_responses[15].json()

{'@meta': {'operation': 'TitleBusinessV2',
  'requestId': 'dce3eb59-99fd-4458-9a7b-69e1e623f75e',
  'serviceTimeMs': 3.670819},
 'resource': {'@type': 'imdb.api.title.v2.business',
  'id': '/title/tt0429493/',
  'title': 'The A-Team',
  'titleType': 'movie',
  'year': 2010,
  'budget': {'amount': 110000000, 'currency': 'USD'},
  'gross': {'aggregations': [{'area': 'XWW',
     'areaName': 'World-wide',
     'total': {'amount': 177238796, 'currency': 'USD'}},
    {'area': 'XNDOM',
     'areaName': 'International',
     'total': {'amount': 100016697, 'currency': 'USD'}},
    {'area': 'XR2',
     'areaName': 'Russia and Commonwealth of Independent States',
     'total': {'amount': 3728526, 'currency': 'USD'}},
    {'area': 'XKN',
     'areaName': 'East Africa',
     'total': {'amount': 25579, 'currency': 'USD'}}],
   'regional': [{'region': 'XDOM',
     'regionName': 'Domestic',
     'total': {'amount': 77222099, 'currency': 'USD'}},
    {'region': 'GB',
     'regionName': 'United Kingdom'

In [165]:
#write function that takes a business reponse and returns dictionary of values
def prepare_bus_response(response):
    resp_json=response.json()
    if 'resource' in resp_json.keys():
        budg_dict={}
        resp_json=resp_json['resource']
        if 'id' in resp_json.keys():
            budg_dict['tconst']=resp_json['id'][7:16]
        if 'budget' in resp_json.keys():
            budg_dict['budget']=resp_json['budget']
        if 'gross' in resp_json.keys():
            if 'aggregations' in resp_json['gross'].keys():
                gross_agg=resp_json['gross']['aggregations']
                for area in gross_agg:
                    if area['area']=='XWW':
                        budg_dict['ww_gross']=area['total']['amount']
                        budg_dict['ww_currency']=area['total']['currency']
                    if area['area']=='XNDOM':
                        budg_dict['int_gross']=area['total']['amount']
                        budg_dict['int_currency']=area['total']['currency']
            if 'regional' in resp_json['gross']:
                gross_region=resp_json['gross']['regional']
                for region in gross_region:
                    if region['region']=='XDOM':
                        budg_dict['domestic_gross']=region['total']['amount']
                        budg_dict['domestic_currency']=region['total']['currency']
        return budg_dict

In [162]:
test=bus_responses[1].json()
test.keys()

dict_keys(['@meta', 'resource'])

In [166]:
business_dicts=[]
for r in bus_responses:
    if r.status_code==200:
        dictionary=prepare_bus_response(r)
        business_dicts.append(dictionary)
business_dicts[:5]

[None,
 {'tconst': 'tt0312305', 'budget': {'amount': 20000000.0, 'currency': 'USD'}},
 None,
 {'tconst': 'tt0326965',
  'budget': {'amount': 1000000.0, 'currency': 'USD'},
  'ww_gross': 30158,
  'ww_currency': 'USD',
  'domestic_gross': 30158,
  'domestic_currency': 'USD'},
 None]

In [167]:
business_dicts_notNa=[x for x in business_dicts if x != None]
len(business_dicts_notNa)

1615

There are only 1615 films that have business info. Not all of those will have both budget and gross info. We will do a left join with the df of movies in the last 10 years from the US. We will also try to use other data sources to get business info that may not be available in IMDB.

In [168]:
imdb_bus_df=pd.DataFrame(business_dicts_notNa)
imdb_bus_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1615 entries, 0 to 1614
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   tconst             1615 non-null   object 
 1   budget             1465 non-null   object 
 2   ww_gross           362 non-null    float64
 3   ww_currency        362 non-null    object 
 4   domestic_gross     324 non-null    float64
 5   domestic_currency  324 non-null    object 
 6   int_gross          226 non-null    float64
 7   int_currency       226 non-null    object 
dtypes: float64(3), object(5)
memory usage: 101.1+ KB


In [175]:
#check for duplicates
duplicates=imdb_bus_df.duplicated(subset='tconst',keep=False)
imdb_bus_df.loc[duplicates]

,tconst,budget,ww_gross,ww_currency,domestic_gross,domestic_currency,int_gross,int_currency
184,tt1149293,"{'amount': 200000.0, 'currency': 'USD'}",NaN,NaN,NaN,NaN,NaN,NaN
185,tt1149293,"{'amount': 200000.0, 'currency': 'USD'}",NaN,NaN,NaN,NaN,NaN,NaN
777,tt1578265,"{'amount': 1500000.0, 'currency': 'USD'}",NaN,NaN,NaN,NaN,NaN,NaN
778,tt1578265,"{'amount': 1500000.0, 'currency': 'USD'}",NaN,NaN,NaN,NaN,NaN,NaN


In [176]:
imdb_bus_df=imdb_bus_df.drop_duplicates(subset='tconst')

In [177]:
imdb_10_US_v1=imdb_10_US.merge(imdb_bus_df,how='left',on='tconst')
imdb_10_US_v1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4166 entries, 0 to 4165
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   tconst             4166 non-null   object 
 1   primary_title      4166 non-null   object 
 2   original_title     4166 non-null   object 
 3   start_year         4166 non-null   int64  
 4   runtime_minutes    3684 non-null   float64
 5   genres             4096 non-null   object 
 6   country            4166 non-null   object 
 7   budget             1465 non-null   object 
 8   ww_gross           362 non-null    float64
 9   ww_currency        362 non-null    object 
 10  domestic_gross     324 non-null    float64
 11  domestic_currency  324 non-null    object 
 12  int_gross          226 non-null    float64
 13  int_currency       226 non-null    object 
dtypes: float64(4), int64(1), object(9)
memory usage: 488.2+ KB


Before I try to get more financial info from other sources, I'll pull MPAA ratings and release dates from the overview detail endpoint of the IMDB API and merge them with the above dataframe.

In [178]:
imdb_overview_url='https://imdb8.p.rapidapi.com/title/get-overview-details'
overview_responses=imdb_requests_byid(ids_10_US,imdb_overview_url)
overview_responses[:5]

[<Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>]

In [187]:
overview_responses[6].json()

{'id': '/title/tt0393049/',
 'title': {'@type': 'imdb.api.title.title',
  'id': '/title/tt0393049/',
  'image': {'height': 947,
   'id': '/title/tt0393049/images/rm679077376',
   'url': 'https://m.media-amazon.com/images/M/MV5BOTcwOTIyNTEzMF5BMl5BanBnXkFtZTgwMDE0MTM2NzE@._V1_.jpg',
   'width': 627},
  'runningTimeInMinutes': 98,
  'title': "Anderson's Cross",
  'titleType': 'movie',
  'year': 2010},
 'certificates': {'US': [{'certificate': 'R',
    'certificateNumber': 49744,
    'ratingReason': 'Rated R for sexual content and language',
    'ratingsBody': 'MPAA',
    'country': 'US'}]},
 'ratings': {'canRate': True, 'rating': 5.2, 'ratingCount': 145},
 'genres': ['Comedy', 'Drama', 'Romance'],
 'releaseDate': '2010-05-20',
 'plotSummary': {'author': 'Anonymous',
  'id': '/title/tt0393049/plot/ps0075747',
  'text': "Nick Anderson, Kevin Daniels, and Tracey Green do everything together. They are the best of friends, and yet they couldn't be more different. Neighbors from adolescence, th

In [202]:
#write function to pull out data from overview responses and return df

def prepare_overview_responses(responses):
    list_dict=[]
    for r in responses:
        if r.status_code==200:
            ov_dict={}
            r_json=r.json()
            if 'id' in r_json.keys():
                ov_dict['tconst']=r_json['id'][7:16]
            if 'certificates' in r_json.keys():
                if 'US' in r_json['certificates'].keys():
                    ov_dict['mpaa_rating']=r_json['certificates']['US'][0]['certificate']
                    #for c in r_json['certificates']['US']:
                        #print(c)
                        #if c['country']=='US':
                         #   ov_dict['mpaa_rating']=c['certificate']
            if 'releaseDate' in r_json.keys():
                ov_dict['release_date']=r_json['releaseDate']
            list_dict.append(ov_dict)
    return pd.DataFrame(list_dict)
                

In [203]:
overview_df=prepare_overview_responses(overview_responses)
overview_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4162 entries, 0 to 4161
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   tconst        4162 non-null   object
 1   release_date  2993 non-null   object
 2   mpaa_rating   1565 non-null   object
dtypes: object(3)
memory usage: 97.7+ KB


In [204]:
imdb_10_US_v2=imdb_10_US_v1.merge(overview_df,how='left',on='tconst')
imdb_10_US_v2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4174 entries, 0 to 4173
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   tconst             4174 non-null   object 
 1   primary_title      4174 non-null   object 
 2   original_title     4174 non-null   object 
 3   start_year         4174 non-null   int64  
 4   runtime_minutes    3690 non-null   float64
 5   genres             4104 non-null   object 
 6   country            4174 non-null   object 
 7   budget             1469 non-null   object 
 8   ww_gross           362 non-null    float64
 9   ww_currency        362 non-null    object 
 10  domestic_gross     324 non-null    float64
 11  domestic_currency  324 non-null    object 
 12  int_gross          226 non-null    float64
 13  int_currency       226 non-null    object 
 14  release_date       3001 non-null   object 
 15  mpaa_rating        1573 non-null   object 
dtypes: float64(4), int64(1),

### Use TMDB API to gather more data ###

In [244]:
#try to find budget for movies that don't have the data
null_bool=imdb_10_US_v2.isnull()
data_columns=['budget','ww_gross','domestic_gross','int_gross','release_date','mpaa_rating']
miss_data_tconsts=[]
for d in data_columns:
    miss_df=imdb_10_US_v2[null_bool[d]]
    imdb_ids=miss_df['tconst']
    miss_data_tconsts.append(imdb_ids)

#merge dataframes listing all tconst for missing values
miss_data_ids=miss_data_tconsts[0].append(miss_data_tconsts[1])
for d in miss_data_tconsts[2:]:
    miss_data_ids=miss_data_ids.append(d)
miss_data_ids=miss_data_ids.drop_duplicates()
miss_data_ids.describe()
imdb_id_missing=miss_data_ids.tolist()

In [207]:
import tmdbsimple as tmdb

In [210]:
   
keys = get_keys("/Users/morgandj/.secret/tmdb_creds.json")
tmdb_key=keys['key']
tmdb_token=keys['read_access_token']
tmdb.API_KEY=tmdb_key

In [216]:

find=tmdb.Find()
response=find.id(query='tt0230212')
search.results

TypeError: 'int' object is not callable

In [245]:
#Loop through the list of imdb ids and make a request to find the tmdb id
#save both in a dictionary, turn into df so it can be merged

for i in imdb_id_missing:
    tmdb_url='https://api.themoviedb.org/3/find/'+ i + '?api_key=' + tmdb_key + '&language=en-US&external_source=imdb_id'
    resp=requests.get(tmdb_url)
    r_json=resp.json()
    if r_json['movie_results']!=[]:
        id_dict={}
        tmdb_id=r_json['movie_results'][0]['id']
        id_dict['tconst']=i
        id_dict['tmdb_id']=tmdb_id
        test_resp.append(id_dict)
tmdb_id_df=pd.DataFrame(test_resp)
tmdb_id_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1689 entries, 0 to 1688
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   tconst   1689 non-null   object
 1   tmdb_id  1689 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 26.5+ KB


In [247]:
#merge tmdb_ids with imdb df
imdb_10_US_v2=imdb_10_US_v2.merge(tmdb_id_df,how='left',on='tconst')
imdb_10_US_v2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4180 entries, 0 to 4179
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   tconst             4180 non-null   object 
 1   primary_title      4180 non-null   object 
 2   original_title     4180 non-null   object 
 3   start_year         4180 non-null   int64  
 4   runtime_minutes    3696 non-null   float64
 5   genres             4110 non-null   object 
 6   country            4180 non-null   object 
 7   budget             1469 non-null   object 
 8   ww_gross           362 non-null    float64
 9   ww_currency        362 non-null    object 
 10  domestic_gross     324 non-null    float64
 11  domestic_currency  324 non-null    object 
 12  int_gross          226 non-null    float64
 13  int_currency       226 non-null    object 
 14  release_date       3007 non-null   object 
 15  mpaa_rating        1576 non-null   object 
 16  tmdb_id            1698 

Budget, genre, and release date are all available in the details endpoint. I will pull this data for all 1698 movies first, then add missing data where I can.

In [251]:
tmdb_id_list=imdb_10_US_v2['tmdb_id'].tolist()
tmdb_det_jsons=[]

for i in tmdb_id_list:
    details_url='https://api.themoviedb.org/3/movie/{}?api_key={}&language=en-US'.\
            format(i,tmdb_key)
    det_resp=requests.get(details_url)
    tmdb_det_jsons.append(det_resp.json())
tmdb_det_jsons

[{'success': False,
  'status_code': 34,
  'status_message': 'The resource you requested could not be found.'},
 {'adult': False,
  'backdrop_path': None,
  'belongs_to_collection': None,
  'budget': 0,
  'genres': [{'id': 10751, 'name': 'Family'},
   {'id': 16, 'name': 'Animation'},
   {'id': 878, 'name': 'Science Fiction'}],
  'homepage': 'http://www.qqthemovie.com/',
  'id': 23738,
  'imdb_id': 'tt0312305',
  'original_language': 'en',
  'original_title': 'Quantum Quest: A Cassini Space Odyssey',
  'overview': 'Dave, a young photon, who is forced out of the Sun on a journey of discovery. He must get to the Cassini Space Craft and save it from the forces of the Void, only Dave does not know exactly what he must do or even where the Cassini Space craft is or what it is. The forces of the Void must destroy Cassini before it sends its great discoveries back to Earth and Dave is the only thing that stands between them and victory.',
  'popularity': 2.558,
  'poster_path': None,
  'produc

In [292]:
#organizes responses into list of dictionaries w/ budget, release date, and genres

def return_det_dict(resp_json):
    json_keys=resp_json.keys()
    if 'success' not in json_keys:
        r_dict={}
        if 'id' in json_keys:
            r_dict['tmdb_id']=resp_json['id']
        if 'budget' in json_keys:
            if resp_json['budget']>0:
                r_dict['budget_tmdb']=resp_json['budget']
        if 'release_date' in json_keys:
            r_dict['release_date_tmdb']=resp_json['release_date']
        if 'genres' in json_keys:
            r_dict['genres_tmdb']=[x['name'] for x in resp_json['genres']]
        return r_dict
            

In [293]:
#organize responses and create df with info for all tmdb_ids
tmdb_list=[]
for r in tmdb_det_jsons:
    dictionary=return_det_dict(r)
    if dictionary != None:
        tmdb_list.append(dictionary)
        
tmdb_df=pd.DataFrame(tmdb_list)
tmdb_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1698 entries, 0 to 1697
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   tmdb_id            1698 non-null   int64  
 1   release_date_tmdb  1698 non-null   object 
 2   genres_tmdb        1698 non-null   object 
 3   budget_tmdb        234 non-null    float64
dtypes: float64(1), int64(1), object(2)
memory usage: 53.2+ KB


Now we will use the release dates endpoint from tmdb to get the MPAA ratings. To reduce the number of API calls, I will only query those tmdb_ids where rating info is not already known. First, pull out these ids, then query, organize into df and merge with tmdb_df.

In [262]:
tmdb_avail=imdb_10_US_v2[imdb_10_US_v2['tmdb_id'].notnull()]
missing_mpaa_tmdb=tmdb_avail[tmdb_avail['mpaa_rating'].isnull()]['tmdb_id'].tolist()
missing_mpaa_tmdb[:5]

[23738.0, 214026.0, 214026.0, 200094.0, 200094.0]

In [265]:
tmdb_mpaa_jsons=[]

for i in missing_mpaa_tmdb:
    mpaa_url='https://api.themoviedb.org/3/movie/{}/release_dates?api_key={}'.\
            format(i,tmdb_key)
    mpaa_resp=requests.get(mpaa_url)
    tmdb_mpaa_jsons.append(mpaa_resp.json())
tmdb_mpaa_jsons[100:115]

[{'id': 457988,
  'results': [{'iso_3166_1': 'US',
    'release_dates': [{'certification': '',
      'iso_639_1': 'en',
      'release_date': '2010-02-05T00:00:00.000Z',
      'type': 3}]}]},
 {'id': 186338,
  'results': [{'iso_3166_1': 'US',
    'release_dates': [{'certification': '',
      'iso_639_1': '',
      'note': '',
      'release_date': '2010-01-01T00:00:00.000Z',
      'type': 3}]}]},
 {'id': 410445,
  'results': [{'iso_3166_1': 'US',
    'release_dates': [{'certification': '',
      'iso_639_1': 'en',
      'release_date': '2010-07-01T00:00:00.000Z',
      'type': 4}]}]},
 {'id': 344722,
  'results': [{'iso_3166_1': 'US',
    'release_dates': [{'certification': '',
      'iso_639_1': '',
      'note': '',
      'release_date': '2010-10-10T00:00:00.000Z',
      'type': 3}]}]},
 {'id': 58550,
  'results': [{'iso_3166_1': 'JP',
    'release_dates': [{'certification': '',
      'iso_639_1': '',
      'note': '',
      'release_date': '2010-03-27T00:00:00.000Z',
      'type': 3

In [273]:
#pull out id and MPAA rating and make a df

mpaa_list=[]

for j in tmdb_mpaa_jsons:
    dictionary={}
    if 'results' in j.keys():
        for r in j['results']:
            if r['iso_3166_1']=='US':
                cert=r['release_dates'][0]['certification']
        if cert != '':
            dictionary['tmdb_id']=j['id']
            dictionary['mpaa_rating_tmdb']=cert
            mpaa_list.append(dictionary)
tmdb_mpaa_df=pd.DataFrame(mpaa_list)
tmdb_mpaa_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82 entries, 0 to 81
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   tmdb_id           82 non-null     int64 
 1   mpaa_rating_tmdb  82 non-null     object
dtypes: int64(1), object(1)
memory usage: 1.4+ KB


In [278]:
tmdb_mpaa_df.drop_duplicates(subset='tmdb_id')
tmdb_mpaa_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82 entries, 0 to 81
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   tmdb_id           82 non-null     int64 
 1   mpaa_rating_tmdb  82 non-null     object
dtypes: int64(1), object(1)
memory usage: 1.4+ KB


Now add the new tmdb data back into original dataframe. Only add data where it is currently null.

In [283]:
master_movies=imdb_10_US_v2.drop_duplicates(subset='tconst')

In [285]:
#Add MPAA ratings
master_movies=master_movies.merge(tmdb_mpaa_df,how='left',on='tmdb_id')
master_movies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4164 entries, 0 to 4163
Data columns (total 18 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   tconst             4164 non-null   object 
 1   primary_title      4164 non-null   object 
 2   original_title     4164 non-null   object 
 3   start_year         4164 non-null   int64  
 4   runtime_minutes    3683 non-null   float64
 5   genres             4094 non-null   object 
 6   country            4164 non-null   object 
 7   budget             1463 non-null   object 
 8   ww_gross           362 non-null    float64
 9   ww_currency        362 non-null    object 
 10  domestic_gross     324 non-null    float64
 11  domestic_currency  324 non-null    object 
 12  int_gross          226 non-null    float64
 13  int_currency       226 non-null    object 
 14  release_date       2991 non-null   object 
 15  mpaa_rating        1561 non-null   object 
 16  tmdb_id            1685 

In [288]:
master_movies['mpaa_rating']=master_movies['mpaa_rating'].fillna(master_movies['mpaa_rating_tmdb'])
master_movies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4164 entries, 0 to 4163
Data columns (total 18 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   tconst             4164 non-null   object 
 1   primary_title      4164 non-null   object 
 2   original_title     4164 non-null   object 
 3   start_year         4164 non-null   int64  
 4   runtime_minutes    3683 non-null   float64
 5   genres             4094 non-null   object 
 6   country            4164 non-null   object 
 7   budget             1463 non-null   object 
 8   ww_gross           362 non-null    float64
 9   ww_currency        362 non-null    object 
 10  domestic_gross     324 non-null    float64
 11  domestic_currency  324 non-null    object 
 12  int_gross          226 non-null    float64
 13  int_currency       226 non-null    object 
 14  release_date       2991 non-null   object 
 15  mpaa_rating        1643 non-null   object 
 16  tmdb_id            1685 

In [291]:
master_movies.drop(labels=['mpaa_rating_tmdb'],axis=1,inplace=True)
master_movies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4164 entries, 0 to 4163
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   tconst             4164 non-null   object 
 1   primary_title      4164 non-null   object 
 2   original_title     4164 non-null   object 
 3   start_year         4164 non-null   int64  
 4   runtime_minutes    3683 non-null   float64
 5   genres             4094 non-null   object 
 6   country            4164 non-null   object 
 7   budget             1463 non-null   object 
 8   ww_gross           362 non-null    float64
 9   ww_currency        362 non-null    object 
 10  domestic_gross     324 non-null    float64
 11  domestic_currency  324 non-null    object 
 12  int_gross          226 non-null    float64
 13  int_currency       226 non-null    object 
 14  release_date       2991 non-null   object 
 15  mpaa_rating        1643 non-null   object 
 16  tmdb_id            1685 

Now do the same for release_date, budget, and genre.

In [294]:
master_movies=master_movies.merge(tmdb_df,how='left',on='tmdb_id')
master_movies['budget']=master_movies['budget'].fillna(master_movies['budget_tmdb'])
master_movies['release_date']=master_movies['release_date'].fillna(master_movies['release_date_tmdb'])
master_movies['genres']=master_movies['genres'].fillna(master_movies['genres_tmdb'])
master_movies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4181 entries, 0 to 4180
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   tconst             4181 non-null   object 
 1   primary_title      4181 non-null   object 
 2   original_title     4181 non-null   object 
 3   start_year         4181 non-null   int64  
 4   runtime_minutes    3697 non-null   float64
 5   genres             4125 non-null   object 
 6   country            4181 non-null   object 
 7   budget             1487 non-null   object 
 8   ww_gross           362 non-null    float64
 9   ww_currency        362 non-null    object 
 10  domestic_gross     324 non-null    float64
 11  domestic_currency  324 non-null    object 
 12  int_gross          226 non-null    float64
 13  int_currency       226 non-null    object 
 14  release_date       3245 non-null   object 
 15  mpaa_rating        1659 non-null   object 
 16  tmdb_id            1702 

In [296]:
master_movies.drop(labels=['release_date_tmdb','genres_tmdb','budget_tmdb'],
                  axis=1,inplace=True)

In [306]:
master_movies=master_movies.drop_duplicates(subset='tconst')
master_movies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4162 entries, 0 to 4180
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   tconst             4162 non-null   object 
 1   primary_title      4162 non-null   object 
 2   original_title     4162 non-null   object 
 3   start_year         4162 non-null   int64  
 4   runtime_minutes    3681 non-null   float64
 5   genres             4106 non-null   object 
 6   country            4162 non-null   object 
 7   budget             1481 non-null   object 
 8   ww_gross           362 non-null    float64
 9   ww_currency        362 non-null    object 
 10  domestic_gross     324 non-null    float64
 11  domestic_currency  324 non-null    object 
 12  int_gross          226 non-null    float64
 13  int_currency       226 non-null    object 
 14  release_date       3226 non-null   object 
 15  mpaa_rating        1641 non-null   object 
 16  tmdb_id            1683 

### Merge Data Using Titles ###

Now that we've gotten as much data by id numbers from IMDB and TMDB as possible, we'll try to combine some of the provided data using titles.

In [320]:
#merge tn_movie_budgets on movie title to get more release dates, budget, gross data
tn_df=tables['tn_movie_budgets_csv_gz']
#tn_df.drop(labels=['id'],axis=1,inplace=True)

master_movies_prim=master_movies.merge(tn_df,how='left',
                                  left_on='primary_title',right_on='movie')
master_movies_prim.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4175 entries, 0 to 4174
Data columns (total 23 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   tconst             4175 non-null   object 
 1   primary_title      4175 non-null   object 
 2   original_title     4175 non-null   object 
 3   start_year         4175 non-null   int64  
 4   runtime_minutes    3692 non-null   float64
 5   genres             4119 non-null   object 
 6   country            4175 non-null   object 
 7   budget             1489 non-null   object 
 8   ww_gross           369 non-null    float64
 9   ww_currency        369 non-null    object 
 10  domestic_gross_x   331 non-null    float64
 11  domestic_currency  331 non-null    object 
 12  int_gross          232 non-null    float64
 13  int_currency       232 non-null    object 
 14  release_date_x     3237 non-null   object 
 15  mpaa_rating        1651 non-null   object 
 16  tmdb_id            1687 

In [322]:
master_movies_prim['budget']=master_movies_prim['budget'].fillna(master_movies_prim['production_budget'])
master_movies_prim['ww_gross']=master_movies_prim['ww_gross'].fillna(master_movies_prim['worldwide_gross'])
master_movies_prim['domestic_gross_x']=master_movies_prim['domestic_gross_x'].fillna(master_movies_prim['domestic_gross_y'])
master_movies_prim.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4175 entries, 0 to 4174
Data columns (total 23 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   tconst             4175 non-null   object 
 1   primary_title      4175 non-null   object 
 2   original_title     4175 non-null   object 
 3   start_year         4175 non-null   int64  
 4   runtime_minutes    3692 non-null   float64
 5   genres             4119 non-null   object 
 6   country            4175 non-null   object 
 7   budget             1528 non-null   object 
 8   ww_gross           430 non-null    object 
 9   ww_currency        369 non-null    object 
 10  domestic_gross_x   393 non-null    object 
 11  domestic_currency  331 non-null    object 
 12  int_gross          232 non-null    float64
 13  int_currency       232 non-null    object 
 14  release_date_x     3237 non-null   object 
 15  mpaa_rating        1651 non-null   object 
 16  tmdb_id            1687 

In [313]:
master_movies_orig=master_movies.merge(tn_df,how='left',
                                       left_on='original_title',right_on='movie')
master_movies_orig.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4176 entries, 0 to 4175
Data columns (total 22 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   tconst             4176 non-null   object 
 1   primary_title      4176 non-null   object 
 2   original_title     4176 non-null   object 
 3   start_year         4176 non-null   int64  
 4   runtime_minutes    3693 non-null   float64
 5   genres             4120 non-null   object 
 6   country            4176 non-null   object 
 7   budget             1489 non-null   object 
 8   ww_gross           369 non-null    float64
 9   ww_currency        369 non-null    object 
 10  domestic_gross_x   331 non-null    float64
 11  domestic_currency  331 non-null    object 
 12  int_gross          232 non-null    float64
 13  int_currency       232 non-null    object 
 14  release_date_x     3238 non-null   object 
 15  mpaa_rating        1652 non-null   object 
 16  tmdb_id            1688 

In [326]:
master_movies_orig['budget']=master_movies_orig['budget'].fillna(master_movies_orig['production_budget'])
master_movies_orig['ww_gross']=master_movies_orig['ww_gross'].fillna(master_movies_orig['worldwide_gross'])
master_movies_orig['domestic_gross_x']=master_movies_orig['domestic_gross_x'].fillna(master_movies_orig['domestic_gross_y'])
master_movies_orig['release_date_x']=master_movies_orig['release_date_x'].fillna(master_movies_orig['release_date_y'])
master_movies_orig.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4176 entries, 0 to 4175
Data columns (total 22 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   tconst             4176 non-null   object 
 1   primary_title      4176 non-null   object 
 2   original_title     4176 non-null   object 
 3   start_year         4176 non-null   int64  
 4   runtime_minutes    3693 non-null   float64
 5   genres             4120 non-null   object 
 6   country            4176 non-null   object 
 7   budget             1529 non-null   object 
 8   ww_gross           433 non-null    object 
 9   ww_currency        369 non-null    object 
 10  domestic_gross_x   395 non-null    object 
 11  domestic_currency  331 non-null    object 
 12  int_gross          232 non-null    float64
 13  int_currency       232 non-null    object 
 14  release_date_x     3247 non-null   object 
 15  mpaa_rating        1652 non-null   object 
 16  tmdb_id            1688 

I checked merging with both primary and original titles, and there were more matches that included previously unknown gross data in original titles, so I will keep that data frame.

In [327]:
master_movies=master_movies_orig.drop(labels=['movie','production_budget',
                                             'domestic_gross_y','worldwide_gross',
                                              'release_date_y'],axis=1)

In [332]:
master_movies[master_movies.duplicated(subset='tconst',keep=False)]

,tconst,primary_title,original_title,start_year,runtime_minutes,genres,country,budget,ww_gross,ww_currency,domestic_gross_x,domestic_currency,int_gross,int_currency,release_date_x,mpaa_rating,tmdb_id
56,tt0800320,Clash of the Titans,Clash of the Titans,2010,106.0,"Action,Adventure,Fantasy",US,"{'amount': 125000000, 'currency': 'USD'}",4.93215e+08,USD,1.63215e+08,USD,330000105.0,USD,2010-04-02,PG-13,NaN
57,tt0800320,Clash of the Titans,Clash of the Titans,2010,106.0,"Action,Adventure,Fantasy",US,"{'amount': 125000000, 'currency': 'USD'}",4.93215e+08,USD,1.63215e+08,USD,330000105.0,USD,2010-04-02,PG-13,NaN
117,tt0955308,Robin Hood,Robin Hood,2010,140.0,"Action,Adventure,Drama",US,"{'amount': 200000000, 'currency': 'USD'}",3.2167e+08,USD,1.0527e+08,USD,216400011.0,USD,2010-05-14,PG-13,NaN
118,tt0955308,Robin Hood,Robin Hood,2010,140.0,"Action,Adventure,Drama",US,"{'amount': 200000000, 'currency': 'USD'}",3.2167e+08,USD,1.0527e+08,USD,216400011.0,USD,2010-05-14,PG-13,NaN
160,tt1014759,Alice in Wonderland,Alice in Wonderland,2010,108.0,"Adventure,Family,Fantasy",US,"{'amount': 200000000, 'currency': 'USD'}",1.02547e+09,USD,3.34191e+08,USD,691277106.0,USD,2010-03-05,PG,NaN
161,tt1014759,Alice in Wonderland,Alice in Wonderland,2010,108.0,"Adventure,Family,Fantasy",US,"{'amount': 200000000, 'currency': 'USD'}",1.02547e+09,USD,3.34191e+08,USD,691277106.0,USD,2010-03-05,PG,NaN
243,tt1106860,Ca$h,Ca$h,2010,108.0,"Crime,Thriller",US,"{'amount': 7000000.0, 'currency': 'USD'}",46488,USD,46488,USD,NaN,NaN,2010-03-26,R,27004.0
244,tt1106860,Ca$h,Ca$h,2010,108.0,"Crime,Thriller",US,"{'amount': 7000000.0, 'currency': 'USD'}",46488,USD,46488,USD,NaN,NaN,2010-03-26,R,27004.0
249,tt1111235,Trance,Trance,2010,83.0,Horror,US,"{'amount': 950000.0, 'currency': 'USD'}","$22,594,052",NaN,"$2,322,593",NaN,NaN,NaN,2010,R,587506.0
250,tt1111235,Trance,Trance,2010,83.0,Horror,US,"{'amount': 950000.0, 'currency': 'USD'}",$0,NaN,$0,NaN,NaN,NaN,2010,R,587506.0


There are 14 duplicates that resulted from the last merge...after reviewing them there are only a couple that could potentially be different movies, so I am just going to drop all duplicates.

In [333]:
master_movies=master_movies.drop_duplicates(subset='tconst')

In [336]:
master_movies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4162 entries, 0 to 4175
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   tconst             4162 non-null   object 
 1   primary_title      4162 non-null   object 
 2   original_title     4162 non-null   object 
 3   start_year         4162 non-null   int64  
 4   runtime_minutes    3681 non-null   float64
 5   genres             4106 non-null   object 
 6   country            4162 non-null   object 
 7   budget             1515 non-null   object 
 8   ww_gross           419 non-null    object 
 9   ww_currency        362 non-null    object 
 10  domestic_gross_x   381 non-null    object 
 11  domestic_currency  324 non-null    object 
 12  int_gross          226 non-null    float64
 13  int_currency       226 non-null    object 
 14  release_date_x     3233 non-null   object 
 15  mpaa_rating        1641 non-null   object 
 16  tmdb_id            1683 

In [330]:
#now merge bom data to see if we can find more box office gross data
bom_df=tables['bom_movie_gross_csv_gz']
bom_df=bom_df.drop(labels=['studio','year'],axis=1)
bom_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3387 entries, 0 to 3386
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           3387 non-null   object 
 1   domestic_gross  3359 non-null   float64
 2   foreign_gross   2037 non-null   object 
dtypes: float64(1), object(2)
memory usage: 79.5+ KB


In [337]:
#merge bom_movie_gross and fill nan values from original data frame
master_movies_o=master_movies.merge(bom_df,how='left',left_on='original_title',right_on='title')
master_movies_o['domestic_gross_x']=master_movies_o['domestic_gross_x'].fillna(master_movies_o['domestic_gross'])
master_movies_o['int_gross']=master_movies_o['int_gross'].fillna(master_movies_o['foreign_gross'])
master_movies_o.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4162 entries, 0 to 4161
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   tconst             4162 non-null   object 
 1   primary_title      4162 non-null   object 
 2   original_title     4162 non-null   object 
 3   start_year         4162 non-null   int64  
 4   runtime_minutes    3681 non-null   float64
 5   genres             4106 non-null   object 
 6   country            4162 non-null   object 
 7   budget             1515 non-null   object 
 8   ww_gross           419 non-null    object 
 9   ww_currency        362 non-null    object 
 10  domestic_gross_x   399 non-null    object 
 11  domestic_currency  324 non-null    object 
 12  int_gross          246 non-null    object 
 13  int_currency       226 non-null    object 
 14  release_date_x     3233 non-null   object 
 15  mpaa_rating        1641 non-null   object 
 16  tmdb_id            1683 

In [339]:
master_movies_o.duplicated(subset='tconst').sum()

0

In [341]:
master_movies=master_movies_o.drop(labels=['title','domestic_gross','foreign_gross'],axis=1)

In [342]:
#save this version of master movies as csv before we start cleaning it just in case
master_movies.to_csv('master_movies.csv')

## Data Preparation

Describe and justify the process for preparing the data for analysis.

***
Runtime: First I will examine the distribution of runtimes to see if there are short films that I may not want to include in the analysis. I would drop them first since they do not need to be cleaned.
***
Box Office Gross: The films need to have either ww_gross or domestic_gross in order to be used for the analysis. I will drop any films that do not have one of these. Then I will examine if I can fill in empty ww_gross by adding international and domestic. I also need to check the currencies and convert the data type to integer.

***
Questions to consider:
* Were there variables you dropped or created?
* How did you address missing values or outliers?
* Why are these choices appropriate given the data and the business problem?
***

In [347]:
master_movies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4162 entries, 0 to 4161
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   tconst             4162 non-null   object 
 1   primary_title      4162 non-null   object 
 2   original_title     4162 non-null   object 
 3   start_year         4162 non-null   int64  
 4   runtime_minutes    3681 non-null   float64
 5   genres             4106 non-null   object 
 6   country            4162 non-null   object 
 7   budget             1515 non-null   object 
 8   ww_gross           419 non-null    object 
 9   ww_currency        362 non-null    object 
 10  domestic_gross_x   399 non-null    object 
 11  domestic_currency  324 non-null    object 
 12  int_gross          246 non-null    object 
 13  int_currency       226 non-null    object 
 14  release_date_x     3233 non-null   object 
 15  mpaa_rating        1641 non-null   object 
 16  tmdb_id            1683 

### runtime_minutes ###

In [350]:
master_movies['runtime_minutes'].describe()

count    3681.000000
mean       83.611247
std        71.317212
min         2.000000
25%        67.000000
50%        85.000000
75%        95.000000
max      3450.000000
Name: runtime_minutes, dtype: float64

I do not want to include the outlier with 3450 min runtime or any other movie longer than 4 hrs since these are outside normal movie lengths. I will drop any rows with runtimes above 240 minutes.

In [352]:
master_movies=master_movies[master_movies['runtime_minutes']<240]
master_movies['runtime_minutes'].describe()

count    3669.000000
mean       81.191060
std        24.289596
min         2.000000
25%        67.000000
50%        84.000000
75%        95.000000
max       238.000000
Name: runtime_minutes, dtype: float64

### ww_gross and domestic_gross ###

In [354]:
master_movies=master_movies[master_movies['ww_gross'].notnull() | 
                            master_movies['domestic_gross_x'].notnull()]
master_movies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 421 entries, 2 to 4044
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   tconst             421 non-null    object 
 1   primary_title      421 non-null    object 
 2   original_title     421 non-null    object 
 3   start_year         421 non-null    int64  
 4   runtime_minutes    421 non-null    float64
 5   genres             418 non-null    object 
 6   country            421 non-null    object 
 7   budget             279 non-null    object 
 8   ww_gross           405 non-null    object 
 9   ww_currency        359 non-null    object 
 10  domestic_gross_x   382 non-null    object 
 11  domestic_currency  322 non-null    object 
 12  int_gross          238 non-null    object 
 13  int_currency       225 non-null    object 
 14  release_date_x     417 non-null    object 
 15  mpaa_rating        375 non-null    object 
 16  tmdb_id            234 no

In [357]:
#examine which types of currencies are provided. I would expect USD since all
#the films originated in the US

print(master_movies['ww_currency'].value_counts())
print(master_movies['domestic_currency'].value_counts())
print(master_movies['int_currency'].value_counts())

USD    359
Name: ww_currency, dtype: int64
USD    322
Name: domestic_currency, dtype: int64
USD    225
Name: int_currency, dtype: int64


All the available currency data shows USD, so we will assume for the few that do not have currency data that they are also USD.

In [377]:
#ensure box office gross data is displayed without sci notation so its easier to read
pd.set_option('display.float_format',lambda x: '%.0f' % x)

In [378]:
#convert ww_gross to float, this column is currently a mix of str and float
#str types need to have '$' and ',' removed
master_movies['ww_gross'].tail(30)
#test=master_movies.loc[2,'ww_gross']
def clean_string(object):
    if type(object)==str:
        return float(object.replace('$','').replace(',',''))
    else: 
        return object

    
master_movies['ww_gross']=master_movies['ww_gross'].apply(clean_string)
master_movies['ww_gross'].describe()


count          405
mean      55253836
std      141238940
min              0
25%          30929
50%         453708
75%       42059111
max     1066970811
Name: ww_gross, dtype: float64

In [393]:
(master_movies['ww_gross']==0.0).sum()

16

In [382]:
#change name of domestic_gross_x to domestic_gross
master_movies=master_movies.rename(columns={'domestic_gross_x': 'domestic_gross'})

In [386]:
#convert strings in domestic gross to float
master_movies['domestic_gross']=master_movies['domestic_gross'].apply(clean_string)

In [387]:
master_movies['domestic_gross'].describe()

count         382
mean     26417191
std      56740417
min             0
25%         24872
50%        387310
75%      26486985
max     415004880
Name: domestic_gross, dtype: float64

In [392]:
(master_movies['domestic_gross']==0.0).sum()

16

I'll remove all movies with no box office income since if they did not have theatrical releases, they do not apply to this analysis.

In [394]:
master_movies=master_movies[master_movies['ww_gross']!=0]
master_movies['ww_gross'].describe()

count          389
mean      57526488
std      143666209
min            131
25%          42273
50%         716580
75%       45636368
max     1066970811
Name: ww_gross, dtype: float64

In [400]:
#convert int_gross to float so we can do calculations with it
master_movies['int_gross']=master_movies['int_gross'].apply(clean_string)
master_movies['int_gross'].describe()

count         238
mean     49318682
std     103190188
min           131
25%        182875
50%       5071910
75%      50107712
max     691277106
Name: int_gross, dtype: float64

In [401]:
#check if int_gross+domestic_gross tends to equal ww_gross where both are available
int_dom=master_movies[master_movies['domestic_gross'].notnull() & 
                      master_movies['int_gross'].notnull() &
                      master_movies['ww_gross'].notnull()]
int_dom['int_dom']=int_dom['domestic_gross']+int_dom['int_gross']
int_dom['diff']=int_dom['ww_gross']-int_dom['int_dom']
int_dom.head()

<ipython-input-401-332988256d45>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  int_dom['int_dom']=int_dom['domestic_gross']+int_dom['int_gross']
<ipython-input-401-332988256d45>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  int_dom['diff']=int_dom['ww_gross']-int_dom['int_dom']


,tconst,primary_title,original_title,start_year,runtime_minutes,genres,country,budget,ww_gross,ww_currency,domestic_gross,domestic_currency,int_gross,int_currency,release_date_x,mpaa_rating,tmdb_id,int_dom,diff
7,tt0398286,Tangled,Tangled,2010,100,"Adventure,Animation,Comedy",US,"{'amount': 260000000, 'currency': 'USD'}",592462816,USD,200821936,USD,391640880,USD,2010-11-24,PG,nan,592462816,0
14,tt0427152,Dinner for Schmucks,Dinner for Schmucks,2010,114,Comedy,US,"{'amount': 69000000, 'currency': 'USD'}",86855739,USD,73026337,USD,13829402,USD,2010-07-30,PG-13,nan,86855739,0
15,tt0429493,The A-Team,The A-Team,2010,117,"Action,Adventure,Thriller",US,"{'amount': 110000000, 'currency': 'USD'}",177238796,USD,77222099,USD,100016697,USD,2010-06-11,PG-13,nan,177238796,0
16,tt0435761,Toy Story 3,Toy Story 3,2010,103,"Adventure,Animation,Comedy",US,"{'amount': 200000000, 'currency': 'USD'}",1066970811,USD,415004880,USD,651965931,USD,2010-06-18,G,nan,1066970811,0
19,tt0446029,Scott Pilgrim vs. the World,Scott Pilgrim vs. the World,2010,112,"Action,Comedy,Fantasy",US,"{'amount': 60000000, 'currency': 'USD'}",48917974,USD,32777690,USD,16140284,USD,2010-08-13,PG-13,nan,48917974,0


In [402]:
int_dom['diff'].describe()

count         192
mean      -166849
std       2352066
min     -32500000
25%             0
50%             0
75%             0
max       1735702
Name: diff, dtype: float64

The inner three quartiles all show no difference between reported ww_gross and the international plus domestic gross. I checked because I don't have enough expertise in the industry to know if there can be differences for any reasons, but this makes me reasonably sure that adding them is a good way to estimate worldwide.

In [ ]:
#get worldwide gross for missing values where domestic and international exist



## Data Modeling
Describe and justify the process for analyzing or modeling the data.

***
Questions to consider:
* How did you analyze or model the data?
* How did you iterate on your initial approach to make it better?
* Why are these choices appropriate given the data and the business problem?
***

In [ ]:
# Here you run your code to model the data


## Evaluation
Evaluate how well your work solves the stated business problem.

***
Questions to consider:
* How do you interpret the results?
* How well does your model fit your data? How much better is this than your baseline model?
* How confident are you that your results would generalize beyond the data you have?
* How confident are you that this model would benefit the business if put into use?
***

## Conclusions
Provide your conclusions about the work you've done, including any limitations or next steps.

***
Questions to consider:
* What would you recommend the business do as a result of this work?
* What are some reasons why your analysis might not fully solve the business problem?
* What else could you do in the future to improve this project?
***